In [ ]:
import numpy as np
import contextlib
import os

import rubin_sim.maf as maf

from u_band_strat import data_dir, det_bands, calc_lbg_density_metric

In [2]:
# Collect data for the table of median depths
depths = dict()
for scale in [1.0, 1.1, 1.2, 1.5]:
    depths[scale] = dict()
    for expt in [30, 38, 45, 60]:
        if expt == 60 and scale == 1.5:
            continue
        depths[scale][expt] = dict()
        for band in "ugrizy":
            depths[scale][expt][band] = dict()
            for year in [1, 10]:
                # Load the dropout band map
                scale_str = str(scale).replace(".", "_")
                with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                    m5 = maf.MetricBundle.load(
                        data_dir
                        / "m5_maps"
                        / f"internal_u_expt{expt}_nscale{scale_str}v3_4_{year}yrs_ExgalM5_{band}.npz",
                    ).metric_values
                depths[scale][expt][band][year] = np.ma.median(m5)

In [ ]:
for scale in depths:
    for expt in depths[scale]:
        row = f"{scale:.1f}x, {expt}s $u$ & {scale * expt / 30:.2f} & "
        for band in "ugrizy":
            row += f"{depths[scale][expt][band][1] - depths[1.0][30][band][1]:>5.2f} "
            if band == "u":
                row += f"{f'({depths[scale][expt][band][10] - depths[1.0][30][band][10]:.2f}':>6}) & "
            else:
                row += f"{f'({depths[scale][expt][band][10] - depths[1.0][30][band][10]: .2f}':>6}) & "
        row = row[:-2] + r"\\"
        print(row)

In [ ]:
fields = ["WFD", "COSMOS", "ECDFS", "ELAIS S1", "XMM-LSS", "EDFS"]

density = {field: dict() for field in fields}
for field in fields:
    name = field.replace(" ", "").replace("-", "")
    for band in "ugriz":
        density[field][band] = dict()
        for year in [1, 10]:
            det_band = det_bands[band]

            # Get names of maps
            if name == "WFD":
                drop = (
                    data_dir / "m5_maps" / f"baseline_v4_0_{year}yrs_ExgalM5_{band}.npz"
                )
                det = (
                    data_dir
                    / "m5_maps"
                    / f"baseline_v4_0_{year}yrs_ExgalM5_{det_band}.npz"
                )
            else:
                drop = (
                    data_dir / "m5_maps_deep" / f"{name}_{year}yrs_ExgalM5_{band}.npz"
                )
                det = (
                    data_dir
                    / "m5_maps_deep"
                    / f"{name}_{year}yrs_ExgalM5_{det_band}.npz"
                )

            # Load maps for dropout and detection bands
            with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                drop = maf.MetricBundle.load(drop).metric_values
                det = maf.MetricBundle.load(det).metric_values

            # Determine number density
            density[field][band][year] = calc_lbg_density_metric(band, drop, det)

In [ ]:
for field in density:
    row = f"{field} & "
    for band in "ugriz":
        if density[field][band][1] >= 10:
            row += f"{float(f'{density[field][band][1]:.2g}'):.0f} "
        else:
            row += str(float(f"{density[field][band][1]:.2g}")) + " "

        if density[field][band][10] >= 10:
            row += f"({float(f'{density[field][band][10]:.2g}'):.0f}) & "
        else:
            row += "(" + str(float(f"{density[field][band][10]:.2g}")) + ") & "

    row = row[:-2] + r"\\"
    print(row)